### Sales Agentic System with Strands SDK - (Orchestration - Worker Pattern) 

This project demonstrates how to build a multi-agent system using the [Strands SDK](https://strandsagents.com/latest/documentation/docs/) to automate the generation and sending of cold sales outreach emails for a fictional company, ComplAI, which offers an AI-powered SOC2 compliance SaaS tool. 


This project provides a practical foundation for building agentic systems for business processes like sales outreach using Strands SDK.

In [ ]:
# Importing necessary components from Strands SDK and other libraries.
from strands.models import BedrockModel
from strands import Agent, tool
from dotenv import load_dotenv
import os
from typing import Dict

# SendGrid imports for email functionality
import sendgrid
from sendgrid.helpers.mail import Email, To, Content
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

In [ ]:
# Load environment variables from a .env file.
# `override=True` ensures that variables in .env take precedence over existing shell variables.
load_dotenv(override=True)

In [ ]:
# Retrieve the AWS region name and sendgrid api key from environment variables.
region_name = os.getenv("AWS_REGION_NAME")
sendgrid_api_key = os.getenv('SENDGRID_API_KEY')

In [ ]:
# This function sends a test email using SendGrid to verify API key and setup.

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
    from_email = Email("iankisali@gmail.com")
    to_email = To("iankisali295@gmail.com")
    content = Content("text/plain", "This is a test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

# Executing test email function
send_test_email()

#### 1. Agentic Workflow
* **Multiple Agent Personas:** It defines and initializes three distinct sales agents—`Professional Sales Agent`, `Engaging Sales Agent`, and `Busy Sales Agent`—each with a unique `system_prompt` to guide their tone and style in generating cold emails (e.g., serious, witty, concise).
* **Email Selection Agent:** A `sales_picker_agent` is introduced to evaluate and select the "best" cold email from those generated by the specialized agents, based on criteria like responsiveness.

In [ ]:
# Define system prompts for different sales agent personas.
# These instructions guide the LLM's behavior and tone.

professional_instructions = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

witty_instructions = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

concise_instructions = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
# Initialize the Bedrock model to be used by the agents.
# We're using Anthropic Claude 3.5 Sonnet, a powerful general-purpose model.
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    region_name=region_name, 
)

In [ ]:
# Initialize different sales agents, each with a unique persona but using the same Bedrock model.

# Professional Sales Agent: Focuses on serious and professional communication.
professional_agent = Agent(
        name="Professional Sales Agent",
        system_prompt=professional_instructions,
        model=bedrock_model
)

# Engaging Sales Agent: Focuses on humorous and witty communication.
witty_agent = Agent(
        name="Engaging Sales Agent",
        system_prompt=witty_instructions,
        model=bedrock_model
)

# Busy Sales Agent: Focuses on concise and direct communication.
concise_agent = Agent(
        name="Busy Sales Agent",
        system_prompt=concise_instructions,
        model=bedrock_model
)

In [ ]:
# Example: Have the professional agent write a cold sales email.
response = professional_agent("Write a cold sales email")

In [ ]:
# Prepare a common message for all agents.
message = "Write a cold sales email"

# Make synchronous calls to all three agents to generate emails.
# Note: These calls will execute sequentially, not in parallel.
results = [
    professional_agent(message),
    witty_agent(message),
    concise_agent(message),
]

In [ ]:
# Print the available attributes/methods of the 'results' list
print(dir(results))

In [ ]:
# Extract the content of the generated emails from the agent responses.
# Each 'result' is an AgentResponse object, and we're accessing its 'message' attribute,
# which typically contains a dictionary with 'content' and 'role'.
outputs = [result.message for result in results]

# Iterate through the extracted email contents and print them.
for output in outputs:
    print(output["content"])

In [ ]:
# sales picker prompt
sales_picker_prompt = "You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only."

# Initialize a 'sales_picker_agent' to select the best email from the generated options.
sales_picker_agent = Agent(
    name="Sales Email Picker",
    system_prompt=sales_picker_prompt,
    model=bedrock_model
)

In [ ]:
message = "Write a cold sales email"
outputs = [result.message for result in results]

In [ ]:
# Concatenate all generated emails into a single string for the sales_picker_agent.
# It joins the content of each email with "Cold sales emails:".
emails = "Cold sales emails:".join([output["content"][0]["text"] for output in outputs])
#print(emails)

# Have the sales_picker_agent select the best email.
best = sales_picker_agent(emails)

# Print the best selected email.
print(f"Best sales email:\n{best.message}")

#### 2. Use of Tools
* **External Function Integration:** The project shows how to integrate external functionalities as tools that agents can invoke. A `send_email` tool is created using SendGrid API to handle the actual sending of emails.
* **Simplified Tool Creation:** Strands SDK's `@tool` decorator simplifies the process of making Python functions callable by agents, eliminating the need for manual JSON boilerplate for tool definitions.

In [ ]:
# Display the professional_agent object.
# In a Jupyter environment, this will show its representation.
professional_agent

### Steps 2 and 3: Tools and Agent Interactions

Strands SDK simplifies tool creation with the `@tool` decorator, eliminating the need for manual JSON boilerplate for tool definitions.

Simply wrap your function with the decorator `@tool`.

In [ ]:
@tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    print(f"\n[Tool Call] Email Agent Engaged.")
    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
    from_email = Email("iankisali@gmail.com")
    to_email = To("iankisali295@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

- This has automatically been converted into a tool, with the boilerplate json created
- The `@tool` decorator handles the necessary internal representation for the LLM to understand and use this function.

In [ ]:
# Display the `send_email` tool object to see its generated metadata.
send_email

#### And you can also convert an Agent into a tool
- This demonstrates the powerful "Agent-as-a-Tool" pattern, allowing hierarchical agent systems.

In [ ]:
@tool
async def professional_agent_tool(query: str) -> str:
    """
    Consults a professional sales expert to get advice, strategies, or information related to sales,
    customer engagement, product benefits, or market approaches.

    Args:
        query (str): The specific question or request for the sales expert.

    Returns:
        str: The advice, strategy, or information provided by the professional sales expert.
    """
    print(f"\n[Tool Call] Delegating to Professional Sales Agent with query: '{query}'")
    response = professional_agent(query)
    print(f"[Tool Call] Professional Sales Agent responded: '{response}'")
    return str(response) # Ensure the response is a string

In [ ]:
# Display the `professional_agent_tool` object to see its generated metadata.
professional_agent_tool

#### 3. Agent Collaboration via Tools and Handoffs
* **Agent-as-a-Tool Pattern:** A key demonstration is the "Agent-as-a-Tool" pattern, where one Strands agent is wrapped in a `@tool` decorated function and then used by another agent. This is shown with `professional_agent_tool`, `engaging_agent_tool`, and `concise_agent_tool`, allowing a higher-level agent to delegate email generation to these specialized agents.
* **Handoff Concept (Orchestration):**(not implemented) A `Sales Manager` agent is designed as an orchestrator. Its `system_prompt` guides it to:
    * Utilize the email-writing agent tools (professional, engaging, concise) to generate multiple email drafts.
    * Select the best email based on its judgment.
    * Conceptually "handoff" to an `Email Manager` agent to format (write subject, convert to HTML) and send the chosen email. Although presented as a handoff, the `Sales Manager` ultimately orchestrates these actions by calling specific tools (`subject_writer_agent_tool`, `html_converter_agent_tool`, `send_html_email`) that perform the formatting and sending steps.

#### So now we can gather all the tools together:

- We're creating tools for each of our three email-writing agents (professional, engaging, concise) and a tool for sending emails.

- A tool for each of our 3 email-writing agents

- And a tool for our function to send emails

In [ ]:
@tool
async def engaging_agent_tool(query: str) -> str:
    """
    Consults an engaging sales expert to get advice, strategies, or information related to sales,
    customer engagement, product benefits, or market approaches.

    Args:
        query (str): The specific question or request for the sales expert.

    Returns:
        str: The advice, strategy, or information provided by the professional sales expert.
    """
    print(f"\n[Tool Call] Delegating to Engaging/Witty Sales Agent with query: '{query}'")
    response = witty_agent(query)
    print(f"[Tool Call] Engaging/Witty Sales Agent responded: '{response}'")
    return str(response)

@tool
async def concise_agent_tool(query: str) -> str:
    """
    Consults a concise/busy sales expert to get advice, strategies, or information related to sales,
    customer engagement, product benefits, or market approaches.

    Args:
        query (str): The specific question or request for the sales expert.

    Returns:
        str: The advice, strategy, or information provided by the professional sales expert.
    """
    print(f"\n[Tool Call] Delegating to Concise/Busy Sales Agent with query: '{query}'")
    response = concise_agent(query)
    print(f"[Tool Call] Concise/Busy Sales Agent responded: '{response}'")
    return str(response)

In [ ]:
# Compile all agent-as-tool wrappers and the email sending tool into a single list.
tools = [professional_agent_tool, engaging_agent_tool, concise_agent_tool, send_email]

# Display the list of compiled tools.
tools

#### And now it's time for our Sales Manager - our planning agent
- The Sales Manager agent will orchestrate the use of the email-writing tools and the email sending tool.

In [ ]:
# sales manager instructions
sales_manager_instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

# Initialize the Sales Manager agent with the defined instructions and tools.
sales_manager = Agent(name="Sales Manager", 
                    system_prompt=sales_manager_instructions, 
                    tools=tools,
                    model=bedrock_model)

In [ ]:
# Define the message for the sales manager.
message = "Send a cold sales email addressed to 'Dear CEO'"

# Execute the sales manager agent with the message.
# This will trigger the agent's reasoning, tool calls, and email selection/sending.
result = sales_manager(message)

In [ ]:
print(result)

#### Handoffs represent a way an agent can delegate to an agent, passing control to it

- Handoffs and Agents-as-tools are similar:

- In both cases, an Agent can collaborate with another Agent

- With tools, control passes back

- With handoffs, control passes across

- This section will likely introduce a different form of agent collaboration compared to the 'Agent-as-a-Tool' pattern, where one agent completely passes control to another to complete a task.

In [ ]:
# Instructions for a subject writer agent.
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

# Instructions for an HTML converter agent.
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# Initialize the Email Subject Writer agent.
subject_writer_agent = Agent(name="Email subject writer", 
                        system_prompt=subject_instructions, 
                        model=bedrock_model)

# This tool wraps the subject_writer agent.
@tool
async def subject_writer_agent_tool(query: str) -> str:
    print(f"\n[Tool Call] Subject writer agent tool engaged")
    response = subject_writer_agent(query)
    print(f"\n[Tool Completed] Subject writer agent tool completed")
    return str(response)

# Initialize the HTML Email Body Converter agent.
html_converter = Agent(name="HTML email body converter", 
                        system_prompt=html_instructions, 
                        model=bedrock_model)

# This tool wraps the html_converter agent.
@tool
async def html_converter_agent_tool(query: str) -> str:
    print(f"\n[Tool Call] HTML Converter agent tool engaged")
    response = html_converter(query)
    print(f"\n[Tool Completed] HTML Converter agent tool completed")
    return str(response)                


In [ ]:
@tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
    from_email = Email("iankisali@gmail.com") 
    to_email = To("iankisali295@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
# Compile the tools for the emailer agent.
tools = [subject_writer_agent_tool, html_converter_agent_tool, send_html_email]

In [ ]:
# Display the list of tools.
tools

In [ ]:
# Email manager instructions - format and send email
email_manager_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

# Initialize the Emailer Agent, which acts as a coordinating agent for email formatting and sending.
emailer_agent = Agent(
    name="Email Manager",
    system_prompt=email_manager_instructions,
    tools=tools,
    model=bedrock_model)

@tool
async def email_agent_tool(query: str) -> str:
    print(f"\n[Tool Call] HTML Converter agent tool engaged")
    response = emailer_agent(query)
    print(f"\n[Tool Completed] HTML Converter agent tool completed")
    return str(response)

#### Now we have 3 tools and 1 handoff
- The `sales_manager` agent will use the `emailer_agent` via a handoff mechanism or by calling it as a tool.

In [ ]:
tools = [subject_writer_agent_tool, html_converter_agent_tool, send_html_email, email_agent_tool]
handoffs = [email_agent_tool]
print(tools)
print(handoffs)

In [ ]:
# Sales manager instructions
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

# Initialize the main Sales Manager agent, which orchestrates the entire process.
sales_manager = Agent(
    name="Sales Manager",
    system_prompt=sales_manager_instructions,
    tools=tools,
    model=bedrock_model)

# Define the primary message for the sales manager.
message = "Send out a cold sales email addressed to Dear CEO from Kisali"

# Execute the sales manager, triggering the entire workflow.
result = sales_manager(message)

In [ ]:
# Print the final result from the sales manager.
print(result)